### 作業目的: 熟練Pytorch Dataset與DataLoader進行資料讀取

本此作業主要會使用[IMDB](http://ai.stanford.edu/~amaas/data/sentiment/)資料集利用Pytorch的Dataset與DataLoader進行
客製化資料讀取。
下載後的資料有分成train與test，因為這份作業目的在讀取資料，所以我們取用train部分來進行練習。
(請同學先行至IMDB下載資料)

### 載入套件

In [ ]:
# Import torch and other required modules
import glob
import torch
import re
import nltk
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_svmlight_file
from nltk.corpus import stopwords

nltk.download('stopwords') #下載stopwords
nltk.download('punkt') #下載word_tokenize需要的corpus

### 探索資料與資料前處理
在train資料中，有分成pos(positive)與neg(negative)，分別為正評價與負評價，此評價即為label。

In [25]:
# 下載資料
from tensorflow.keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(path= "imdb.npz")

# 以nltk stopwords移除贅字，過多的贅字無法提供有用的訊息，也可能影響模型的訓練
#print(f"vocab length before removing stopwords: {len(vocab)}")
#vocab = list(set(vocab).difference(set(stopwords.words('english'))))
#print(f"vocab length after removing stopwords: {len(vocab)}")

# 將字典轉換成dictionary
#vocab_dic = dict(zip(vocab, range(len(vocab))))

vocab = []
for i in range(len(x_train)):
    vocab.extend(x_train[i])

vocab = set(vocab)
vocab_dic = dict(zip(vocab,range(len(vocab))))

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [26]:
# 將資料打包成(x, y)配對，其中x為review的檔案路徑，y為正評(1)或負評(0)
# 這裡將x以檔案路徑代表的原因是讓同學練習不一次將資料全讀取進來，若電腦記憶體夠大(所有資料檔案沒有很大)
# 可以將資料全一次讀取，可以減少在訓練時I/O時間，增加訓練速度

#review_pos = glob.glob("./aclImdb/train/pos/*.txt")
#review_neg = glob.glob("./aclImdb/test/neg/*.txt")
#review_all = review_pos + review_neg
#y = [1]*len(review_pos) + [0]*len(review_neg)

#review_pairs = list(zip(review_all, y))
#print(review_pairs[:2])
#print(f"Total reviews: {len(review_pairs)}")

review_pairs = list(zip(x_train,y_train))

### 建立Dataset與DataLoader讀取資料
這裡我們會需要兩個helper functions，其中一個是讀取資料與清洗資料的函式(load_review)，另外一個是生成詞向量BoW的函式
(generate_bow)

In [27]:
def load_review(review_path):
    
    with open(review_path, 'r') as f:
        review = f.read()
        
    #移除non-alphabet符號、贅字與tokenize
    #review = re.sub('[^a-zA-Z]',' ',review)
    #review = nltk.word_tokenize(review)
    #review = list(set(review).difference(set(stopwords.words('english'))))
    
    return review

In [28]:

def generate_bow(review, vocab_dic):
    bag_vector = np.zeros(len(vocab_dic))
    for word in review:
        if vocab_dic.get(word):
            bag_vector[vocab_dic.get(word)] += 1
            
    return bag_vector

In [32]:
class dataset(Dataset):
    '''custom dataset to load reviews and labels
    Parameters
    ----------
    data_pairs: list
        directory of all review-label pairs
    vocab: list
        list of vocabularies
    '''
    def __init__(self, data_dirs, vocab):
        self.data_dirs = data_dirs
        self.vocab = vocab

    def __len__(self):
        return len(self.data_dirs)

    def __getitem__(self, idx):
        pair = self.data_dirs[idx]
        review = pair[0]
        #review = load_review(review)
        review = generate_bow(review, self.vocab)
        
        return review, pair[1]

In [33]:
custom_dst = dataset(review_pairs, vocab_dic)
custom_dst[10]

(array([ 0.,  0., 37., ...,  0.,  0.,  0.]), 1)

In [40]:
custom_dataloader = DataLoader(dataset=custom_dst, batch_size=4, shuffle=True)#from torch.utils.data import Dataset, DataLoader
next(iter(custom_dataloader))

[tensor([[ 0.,  0.,  7.,  ...,  0.,  0.,  0.],
         [ 0.,  0., 38.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  7.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  2.,  ...,  0.,  0.,  0.]], dtype=torch.float64),
 tensor([1, 0, 0, 0])]

In [39]:
next(iter(custom_dataloader))

[tensor([[ 0.,  0., 17.,  ...,  0.,  0.,  0.],
         [ 0.,  0., 18.,  ...,  0.,  0.,  0.],
         [ 0.,  0., 13.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  8.,  ...,  0.,  0.,  0.]], dtype=torch.float64),
 tensor([1, 1, 1, 0])]